## Notes:
----------

Theory:
We have see a lot of discreapancy with the images, hence for testing puspose we only select images that were last_reviewed after September 2017. We believe that the data atleast reviewed after september should be correct.

1. The accessor photo column had some "nan" values, these were removed.
2. 

In [1]:
import os
import random
import pandas as pd
import numpy as np
# import config
import logging

logging.basicConfig(level=logging.DEBUG, filename="logfile.log", filemode="w", 
                    format="%(asctime)-15s %(levelname)-8s %(message)s")

from ExternalData.bing_search import get_image_address

dump_assesor_images = False

In [2]:
csv_data_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/metadata_last_reviewed_2017_+sept.csv'

parent_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/'
assessor_image_path = os.path.join(parent_path, "input_images", "assessor_images")
aerial_image_path = os.path.join(parent_path, 'input_images','aerial_images')

print('Assessor image path: ', assessor_image_path)
print('Aerial image path: ', aerial_image_path)

Assessor image path:  /Users/sam/All-Program/App-DataSet/HouseClassification/input_images/assessor_images
Aerial image path:  /Users/sam/All-Program/App-DataSet/HouseClassification/input_images/aerial_images


### LOAD METADATA CSV

In [3]:
image_data = pd.read_csv(csv_data_path)
print (image_data.shape)
image_data.head()

(6080, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
0,493,0,500372,IL,Cook,02-12-300-116-0000,1188 N. CHESAPEAKE LANE,NaN,PALATINE,60074.0,Residential,Improved,Townhouse,NaN,2017-10-18,NaN,Likely House,http://www.cookcountyassessor.com/PropertyImag...
1,2230,0,502281,IL,Cook,04-16-404-018-0000,2136 SECOND ST,NaN,NORTHBROOK,600626620.0,Vacant,Land,NaN,NaN,2017-07-06,NaN,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,3731,0,503918,IL,Cook,06-36-307-008-0000,5052 ELM AVE,NaN,HANOVER PARK,601330000.0,Residential,Land,NaN,NaN,2017-07-06,NaN,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
3,5086,0,505343,IL,Cook,08-15-402-025-1062,1115 E ALGONQUIN RD,UNIT 2,ARLINGTON HEIGHTS,NaN,Residential,Condominium,NaN,NaN,2017-07-10,NaN,Likely House,http://www.cookcountyassessor.com/PropertyImag...
4,5106,0,505371,IL,Cook,08-22-200-166-0000,1226 LYNN CT,NaN,MOUNT PROSPECT,NaN,Residential,Improved,Townhouse,Brick,2017-07-06,NaN,Likely House,http://www.cookcountyassessor.com/PropertyImag...


In [4]:
image_data.columns = ['row_id', 'removed', 'property_id', 'state', 'county_name', 'pin',
       'address_line1', 'address_line2', 'address_city', 'address_zip',
       'zoning', 'improvement_level', 'type', 'exterior',
       'last_reviewed_timestamp', 'gone_timestamp', 'indicator',
       'assessor_photo']
image_data.dtypes

row_id                      int64
removed                     int64
property_id                 int64
state                      object
county_name                object
pin                        object
address_line1              object
address_line2              object
address_city               object
address_zip                object
zoning                     object
improvement_level          object
type                       object
exterior                   object
last_reviewed_timestamp    object
gone_timestamp             object
indicator                  object
assessor_photo             object
dtype: object

## Data Type Conversion

In [5]:
image_data['state'] = image_data['state'].astype('str')
image_data['county_name'] = image_data['county_name'].astype('str')
image_data['pin'] = image_data['pin'].astype('str')
image_data['address_line1'] = image_data['address_line1'].astype('str')
image_data['address_line2'] = image_data['address_line2'].astype('str')
image_data['address_city'] = image_data['address_city'].astype('str')
image_data['address_zip'] = image_data['address_zip'].astype('str')
image_data['zoning'] = image_data['zoning'].astype('str')
image_data['improvement_level'] = image_data['improvement_level'].astype('str')
image_data['type'] = image_data['type'].astype('str')
image_data['exterior'] = image_data['exterior'].astype('str')
image_data['last_reviewed_timestamp'] = pd.to_datetime(image_data['last_reviewed_timestamp'])#.astype('str')
image_data['gone_timestamp'] = pd.to_datetime(image_data['gone_timestamp'])#.astype('str')
image_data['indicator'] = image_data['indicator'].astype('str')
image_data['assessor_photo'] = image_data['assessor_photo'].astype('str')

In [6]:
print('Unique row_id', len(np.unique(np.array(image_data['row_id']))))
print('Unique removed', len(np.unique(np.array(image_data['removed']))))
print('Unique property_id', len(np.unique(np.array(image_data['property_id']))))
print('Unique state,', len(np.unique(np.array(image_data['state']))))
print('Unique county_name,', len(np.unique(np.array(image_data['county_name']))))
print('Unique property pin', len(np.unique(np.array(image_data['pin']))))
print('Unique zoning', len(np.unique(np.array(image_data['zoning']))))
print('Unique improvement_level', len(np.unique(np.array(image_data['improvement_level']))))
print('Unique exterior', len(np.unique(np.array(image_data['exterior']))))
print('Unique indicator', len(np.unique(np.array(image_data['indicator']))))
print('Unique assessor_photo', len(np.unique(np.array(image_data['assessor_photo']))))

Unique row_id 6080
Unique removed 1
Unique property_id 6080
Unique state, 1
Unique county_name, 2
Unique property pin 6080
Unique zoning 2
Unique improvement_level 6
Unique exterior 5
Unique indicator 2
Unique assessor_photo 6080


In [7]:
print('Unique improvement_level', np.unique(np.array(image_data['improvement_level'])))
print('Unique exterior', np.unique(np.array(image_data['exterior'])))
print('Unique indicator', np.unique(np.array(image_data['indicator'])))

Unique improvement_level ['Condominium' 'Improved' 'Land' 'Minor Improvement' 'Mixed Use' 'Unknown']
Unique exterior ['Brick' 'Brick/Frame' 'Frame' 'Stucco' 'nan']
Unique indicator ['Likely House' 'Likely Land']


### SELECT DATA SEGREGATED FOR TRAINING 

In [8]:
image_data = image_data[image_data['removed'] == 0]
print (image_data.shape)
image_data.head()

(6080, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
0,493,0,500372,IL,Cook,02-12-300-116-0000,1188 N. CHESAPEAKE LANE,nan,PALATINE,60074.0,Residential,Improved,Townhouse,nan,2017-10-18,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
1,2230,0,502281,IL,Cook,04-16-404-018-0000,2136 SECOND ST,nan,NORTHBROOK,600626620.0,Vacant,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,3731,0,503918,IL,Cook,06-36-307-008-0000,5052 ELM AVE,nan,HANOVER PARK,601330000.0,Residential,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
3,5086,0,505343,IL,Cook,08-15-402-025-1062,1115 E ALGONQUIN RD,UNIT 2,ARLINGTON HEIGHTS,nan,Residential,Condominium,nan,nan,2017-07-10,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
4,5106,0,505371,IL,Cook,08-22-200-166-0000,1226 LYNN CT,nan,MOUNT PROSPECT,nan,Residential,Improved,Townhouse,Brick,2017-07-06,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### LAND DATA
----------------

In [9]:
land_data = image_data[image_data['indicator'] == 'Likely Land']
print (land_data.shape)
land_data.head(3)

(1715, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
1,2230,0,502281,IL,Cook,04-16-404-018-0000,2136 SECOND ST,nan,NORTHBROOK,600626620.0,Vacant,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,3731,0,503918,IL,Cook,06-36-307-008-0000,5052 ELM AVE,nan,HANOVER PARK,601330000.0,Residential,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
12,13654,0,514974,IL,Cook,12-21-210-072-0000,9690 IVANHOE AVE,nan,SCHILLER PARK,601762415.0,Vacant,Land,nan,nan,2017-07-10,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### HOUSE DATA
--------------

In [10]:
house_data = image_data[image_data['indicator'] == 'Likely House']
print (house_data.shape)
house_data.head(3)

(4365, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
0,493,0,500372,IL,Cook,02-12-300-116-0000,1188 N. CHESAPEAKE LANE,nan,PALATINE,60074.0,Residential,Improved,Townhouse,nan,2017-10-18,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
3,5086,0,505343,IL,Cook,08-15-402-025-1062,1115 E ALGONQUIN RD,UNIT 2,ARLINGTON HEIGHTS,nan,Residential,Condominium,nan,nan,2017-07-10,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
4,5106,0,505371,IL,Cook,08-22-200-166-0000,1226 LYNN CT,nan,MOUNT PROSPECT,nan,Residential,Improved,Townhouse,Brick,2017-07-06,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


### UNKNOWN DATA
-------------

In [11]:
unknown_data = image_data[image_data['indicator'] == 'Unknown']
print (unknown_data.shape)
unknown_data.head(3)

(0, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo


# LOAD DATA - ASSESSOR IMAGES
------------

In [12]:
import urllib.request
import requests
import os


def upload_assessor_data(dataIN, dump_path, how_many_images):
    if how_many_images > len(dataIN):
        raise ValueError ('The random sample cannot exceed the dataIN length')
    random.seed(8792)
    random_indices = random.sample(range(1, len(dataIN)+1), how_many_images)
    random_indices = np.sort(np.array(random_indices, dtype='int32'))    
    dataIN.loc[:,'row_num'] = np.arange(1, len(dataIN)+1)
    dataIN = dataIN[dataIN['row_num'].isin(random_indices)]
    print ('Total images to be loaded is:', dataIN.shape)
    i = 0
    j = 0
    for  row_id, pin, pic_url in np.array(dataIN[['row_id', 'pin', 'assessor_photo']]):
#         print (pic_url)
        response = requests.get(pic_url)
        if response.status_code == 200:
            imagepath = os.path.join(dump_path, '%s.jpg'%(str(pin)))
            with open(imagepath, 'wb') as f:
                f.write(response.content)
            i+=1
        else:
            j += 1
        
        b = "TOTAL IMAGES PARSED ======== %s"
        print (b%(i), end="\r")
    print ('')
    print ("TOTAL FAULTY IMAGES ==== ", j)


#### LOAD LAND IMAGES

In [13]:
if dump_assesor_images:
    assessor_land_dump_path = os.path.join(assessor_image_path, 'land')
    upload_assessor_data(dataIN=land_data, dump_path=assessor_land_dump_path, how_many_images=1100)

#### LOAD HOUSE IMAGES

In [14]:
if dump_assesor_images:
    assessor_house_dump_path = os.path.join(assessor_image_path, 'house')
    upload_assessor_data(dataIN=house_data, dump_path=assessor_house_dump_path, how_many_images=1100)

## REMOVE NOT-AVAILABLE IMAGES
------------------

In [15]:
from scipy import misc
def remove_not_available_data(image_path):
    image_path = [os.path.join(image_path, img) for img in os.listdir(image_path) if img != ".DS_Store"]
    not_available_image_path = '/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/c_image_na.jpg'
    bad_data = misc.imread(not_available_image_path)
    bad_data = misc.imresize(bad_data, [160,160])
    for path in image_path:
        data = misc.imread(path)
        data = misc.imresize(data, [160,160])
        if (bad_data == data).all():
            print ('Found: ', os.path.basename(path).split('.')[0])
            os.remove(path)

#### LAND - REMOVE

In [16]:
if dump_assesor_images:
    assessor_land_dump_path = os.path.join(assessor_image_path, 'land')
    remove_not_available_data(image_path=assessor_land_dump_path)

In [17]:
if dump_assesor_images:
    assessor_house_dump_path = os.path.join(assessor_image_path, 'house')
    remove_not_available_data(image_path=assessor_house_dump_path)

# LOAD DATA - AERIAL IMAGE:
------------

In [18]:
assessor_land_dump_path = os.path.join(assessor_image_path, 'land')
assessor_house_dump_path = os.path.join(assessor_image_path, 'house')

aerial_land_dump_path = os.path.join(aerial_image_path, 'land')
aerial_house_dump_path = os.path.join(aerial_image_path, 'house')

print (assessor_land_dump_path)
print (assessor_house_dump_path)
print (aerial_land_dump_path)
print (aerial_house_dump_path)

/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/assessor_images/land
/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/assessor_images/house
/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/aerial_images/land
/Users/sam/All-Program/App-DataSet/HouseClassification/input_images/aerial_images/house


### Aerial Land

In [19]:
land_pins = [img.split('.')[0] for img in os.listdir(assessor_land_dump_path) if img!=".DS_Store"]
land_data = image_data[image_data['pin'].isin(land_pins)]
print (land_data.shape)
land_data.head(3)

(654, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
1,2230,0,502281,IL,Cook,04-16-404-018-0000,2136 SECOND ST,nan,NORTHBROOK,600626620.0,Vacant,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
2,3731,0,503918,IL,Cook,06-36-307-008-0000,5052 ELM AVE,nan,HANOVER PARK,601330000.0,Residential,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...
15,14488,0,515984,IL,Cook,12-26-413-071-0000,2564N MARWOOD ST,nan,RIVER GROVE,60171.0,Residential,Land,nan,nan,2017-07-06,NaT,Likely Land,http://www.cookcountyassessor.com/PropertyImag...


### Aerial House

In [20]:
house_pins = [img.split('.')[0] for img in os.listdir(assessor_house_dump_path) if img!=".DS_Store"]
house_data = image_data[image_data['pin'].isin(house_pins)]
print (house_data.shape)
house_data.head(3)

(891, 18)


,row_id,removed,property_id,state,county_name,pin,address_line1,address_line2,address_city,address_zip,zoning,improvement_level,type,exterior,last_reviewed_timestamp,gone_timestamp,indicator,assessor_photo
0,493,0,500372,IL,Cook,02-12-300-116-0000,1188 N. CHESAPEAKE LANE,nan,PALATINE,60074.0,Residential,Improved,Townhouse,nan,2017-10-18,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
3,5086,0,505343,IL,Cook,08-15-402-025-1062,1115 E ALGONQUIN RD,UNIT 2,ARLINGTON HEIGHTS,nan,Residential,Condominium,nan,nan,2017-07-10,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...
8,8601,0,509328,IL,Cook,10-24-214-016-0000,1800 GREENLEAF ST,nan,EVANSTON,602021011.0,Residential,Improved,nan,nan,2017-07-10,NaT,Likely House,http://www.cookcountyassessor.com/PropertyImag...


In [21]:
combined_data = pd.concat([land_data, house_data])
get_image_address(dataIN=combined_data, batch_size=100)

[['04-16-404-018-0000' '2136 SECOND ST' 'NORTHBROOK' '600626620.0'
  'Likely Land']
 ['06-36-307-008-0000' '5052 ELM AVE ' 'HANOVER PARK' '601330000.0'
  'Likely Land']
 ['12-26-413-071-0000' '2564N MARWOOD ST' 'RIVER GROVE' '60171.0'
  'Likely Land']
 ['12-29-205-035-0000' '10615 DIVERSEY AVE ' 'MELROSE PARK' 'nan'
  'Likely Land']
 ['12-36-313-018-0000' '1725 N THATCHER AVE ' 'ELMWOOD PARK' '607073545.0'
  'Likely Land']]
